### Autonomous Driving in Airsim: Preparing the data and label softening

In [1]:
import pandas as pd
import numpy as np
import h5py
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import os
import random

# directories of the raw data
RAW_TRAIN_DATA_PATH = 'raw_data/traincombined'
RAW_TEST_DATA_PATH = 'raw_data/test'
RAW_VALID_DATA_PATH = 'raw_data/validation'

# output directory for the preprocessed data
OUTPUT_DATA_PATH= 'preprocessed_data_ls/'


os.chdir("C:/Users/Pascal/AppData/Local/Programs/Python/Python35/Scripts/AppliedDataScience/AirSim_Experiments/label_smoothing")


In [3]:

for PATH in [RAW_TRAIN_DATA_PATH, RAW_TEST_DATA_PATH, RAW_VALID_DATA_PATH]:
    # check if it exists already preprocessed data at the given location, if so skip this iteration
    file = OUTPUT_DATA_PATH + PATH[9:] + ".h5"  # to create /train.h5, /test.h5 , /valid.h5  

    if(os.path.isfile(file) ):
        print(file + " exists already... continue")
        continue
 
    record = pd.read_csv(PATH + '/airsim_rec.txt', sep='\t') 
    
    # the file names of the images 
    image_names = np.array(record["ImageFile"])  
    
    # the steering which correspong to the images
    c_steering = pd.read_csv(PATH + '/airsim_rec.txt', sep='\t')["Steering"]

    
    smooth_steering = np.array([0.]*len(c_steering))
    
    # indices where steering is 1 or -1
    ind_steer_right = np.argwhere(c_steering == 1.).flatten()
    ind_steer_left  = np.argwhere(c_steering == -1.).flatten()

    # apply the label softening sliding window
    smooth_steering[ind_steer_right] = smooth_steering[ind_steer_right]+0.4
    smooth_steering[ind_steer_right+1] = smooth_steering[ind_steer_right+1]+0.2
    smooth_steering[ind_steer_right+2] = smooth_steering[ind_steer_right+2]+0.1
    smooth_steering[ind_steer_right-1] = smooth_steering[ind_steer_right-1]+0.2
    smooth_steering[ind_steer_right-2] = smooth_steering[ind_steer_right-2]+0.1

    smooth_steering[ind_steer_left] = smooth_steering[ind_steer_left]-0.4
    smooth_steering[ind_steer_left+1] = smooth_steering[ind_steer_left+1]-0.2
    smooth_steering[ind_steer_left+2] = smooth_steering[ind_steer_left+2]-0.1
    smooth_steering[ind_steer_left-1] = smooth_steering[ind_steer_left-1]-0.2
    smooth_steering[ind_steer_left-2] = smooth_steering[ind_steer_left-2]-0.1
    
    
    
    
    current_steering = smooth_steering
    
    del record

    # shuffle the data 
    shuffled_indices = list(range(len(image_names)))
    random.shuffle(shuffled_indices)
    image_names      = image_names[shuffled_indices]
    current_steering = current_steering[shuffled_indices]
    last_steering    = last_steering[shuffled_indices]

    
    images = [np.nan]*len(image_names)
    for i, image_name in enumerate(image_names):
        im = Image.open( PATH + "/images/" + image_name)
                
        image_array = np.asarray(im)
        
        # remove alpha channel (it is always zero)
        if len(image_array.shape) == 3 and image_array.shape[2] == 4:
            if (np.all(image_array[:, :, 3] == image_array[0, 0, 3])):
                image_array = image_array[:,:,0:3]

        images[i] = image_array
    


    # create the datasets
    with h5py.File(file, 'w') as output_file:
        
        print("processing: " + file[18:]) # print status message
        
        dset_images           = output_file.create_dataset("image", data=images)
        dset_current_steering = output_file.create_dataset("current_steering", data=current_steering)
        dset_last_steering    = output_file.create_dataset("last_steering", data=last_steering)

# free RAM
del image_names
del current_steering
del last_steering
    

processing: ls/traincombined.h5
processing: ls/test.h5
processing: ls/validation.h5
